### Fashion Mnist using RNN

Start by importing required libraries/packages 

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

c:\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Read data 

In [2]:
mnist=input_data.read_data_sets('../../dataSets/fashionMnist',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataSets/fashionMnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataSets/fashionMnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../dataSets/fashionMnist\t10k-images-idx3-ubyte.gz
Extracting ../../dataSets/fashionMnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Declare some python assignments/constants, like number of epochs, number of output classes, input vector size. 

In [3]:
image_height = 28
image_width = 28
num_time_steps=28
num_units=128
num_input=28
learning_rate=0.001
num_classes=10
minibatch_size=128
num_epochs = 2

Begin building TF Graph, by declaring input and output placeholders

InputPlaceholder a tensor of rank three(3D) of shape batch size, number of time steps and number of inputs dimension. In our case number of time steps is 28, so is number of inputs dimension, 28. Each image with heigth and width being 28 * 28, is unrolled to time steps of 28 and dimension of imput at each time step 28 features(pixels in our case).

OutputPlaceholder : is a tensor of rank two, with shape batch size and number of output classes, which is 10.

In [4]:
X_placeholder=tf.placeholder(tf.float32,[None,num_time_steps,num_input])
Y_placeholder=tf.placeholder(tf.float32,[None,num_classes])

Our LSTM model to tell, which of 10 class the give image belongs we need to consider only the output of last step which is 28th step. Here we delcare the weights and bias for the calculating same.

In [5]:
output_weights=tf.get_variable("Output_Weights", [num_units,num_classes], initializer=tf.initializers.random_normal(seed=12))
output_bias=tf.get_variable("Output_bias", [num_classes], initializer=tf.initializers.random_normal(seed=12))

Formatting inputs: We unstack the input data, so we can feed the data of each steps to its respective LSTM cell

In [6]:
X_unstack=tf.unstack(X_placeholder ,num_time_steps,1)

num_units : is the number of nodes of LSTM cell, Static_rnn is a simplest for of RNN we can use dynamic_rnn nothing stops us. Dynamic is used for input of different length's, in out case the length of input is static , with length of 28, not dynamic. 

The inputs argument accepts list of tensors of shape [batch_size,input_size].The length of this list is the number of time steps through which network is unrolled i.e. each element of this list corresponds to the input at respective time step of our unrolled network.

The output generated by static_rnn is a list of tensors of shape [batch_size,num_units].The length of the list is number of time steps through which network is unrolled i.e. one output tensor for each time step.In this implementation we will only be concerned with output of the final time step as the prediction will be generated when all the rows of an image are supplied to RNN i.e. at the last time step.

In [7]:
lstm_layer=tf.contrib.rnn.BasicLSTMCell(num_units)
outputs,_=tf.contrib.rnn.static_rnn(lstm_layer,X_unstack,dtype=tf.float32)

We consider only the last output, another way to put this is model we are training/building is many to one, which is we feed.

In [8]:
prediction=tf.add(tf.matmul(outputs[-1],output_weights),output_bias)

Now the code takes back its usual for, for calculating loss and runing optimizer, we used for DNN and CNN .. i.e calculate loss with softmax cross entropy , optimizer used is Adama Optimizer with learning rate of 0.0001. 

In [9]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=Y_placeholder))
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(Y_placeholder,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Create a session for each epoch, train will all the data with a minibatch of size 128. :)

In [10]:
m = 60000
num_minibatches = int(m / minibatch_size)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epochs):
        for _ in range(num_minibatches):
            batch_x,batch_y=mnist.train.next_batch(batch_size=minibatch_size)
            batch_x=batch_x.reshape((minibatch_size,num_time_steps,num_input))
            sess.run(opt, feed_dict={X_placeholder: batch_x, Y_placeholder: batch_y})
            
        print("Epoch {Epoch}, training batch accuracy {Accuracy}".format(
            Epoch=epoch, Accuracy=sess.run(accuracy, feed_dict={X_placeholder: batch_x, Y_placeholder: batch_y})))

    
    X_test = mnist.test.images # X_test shape: [num_test, 28*28]
    X_test = X_test.reshape((-1,num_time_steps,num_input))
    y_test = mnist.test.labels
    
    print("Testdata accuracy {CalculatedAccuracy}".format(
        CalculatedAccuracy=accuracy.eval(feed_dict={X_placeholder: X_test, Y_placeholder: y_test})))

Epoch 0, training batch accuracy 0.796875
Epoch 1, training batch accuracy 0.875
Testdata accuracy 0.8314999938011169
